In [1]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.WARN)
import pickle
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import os
from tensorflow.python.client import device_lib

In [2]:
f = open('../Glove/word_embedding_glove', 'rb')
word_embedding = pickle.load(f)
f.close()
word_embedding = word_embedding[: len(word_embedding)-1]

f = open('../Glove/vocab_glove', 'rb')
vocab = pickle.load(f)
f.close()

word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"

f = open("train.pickle", 'rb')
full_data = pickle.load(f)
f.close()

In [3]:
# Model Description
sense_word = 'hard'
model_name = 'basic 2'
model_dir = 'output/' + sense_word + '/' + model_name
save_dir = os.path.join(model_dir, "save/")
log_dir = os.path.join(model_dir, "log")

if not os.path.exists(model_dir):
    os.mkdir(model_dir)
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

In [4]:
# Parameters
mode = 'train'
num_senses = 3
batch_size = 64
vocab_size = len(vocab)
unk_vocab_size = 1
word_emb_size = len(word_embedding[0])
max_sent_size = 200
hidden_size = 100
keep_prob = 0.5
l2_lambda = 0.001
init_lr = 0.001
decay_steps = 5000
decay_rate = 0.96
clip_norm = 1
clipping = True

In [5]:
# MODEL
x = tf.placeholder('int32', [batch_size, max_sent_size], name="x")
y = tf.placeholder('int32', [batch_size], name="y")
x_mask  = tf.placeholder('bool', [batch_size, max_sent_size], name='x_mask') 
is_train = tf.placeholder('bool', [], name='is_train')
word_emb_mat = tf.placeholder('float', [None, word_emb_size], name='emb_mat')
input_keep_prob = tf.cond(is_train,lambda:keep_prob, lambda:tf.constant(1.0))
x_len = tf.reduce_sum(tf.cast(x_mask, 'int32'), 1)

with tf.name_scope("word_embedding"):
    if mode == 'train':
        unk_word_emb_mat = tf.get_variable("word_emb_mat", dtype='float', shape=[unk_vocab_size, word_emb_size], initializer=tf.contrib.layers.xavier_initializer(uniform=True, seed=None, dtype=tf.float32))
    else:
        unk_word_emb_mat = tf.get_variable("word_emb_mat", shape=[unk_vocab_size, word_emb_size], dtype='float')
        
    final_word_emb_mat = tf.concat([word_emb_mat, unk_word_emb_mat], 0)
    Wx = tf.nn.embedding_lookup(final_word_emb_mat, x)  

with tf.variable_scope("lstm1"):
    cell_fw1 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
    cell_bw1 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

    d_cell_fw1 = tf.contrib.rnn.DropoutWrapper(cell_fw1, input_keep_prob=input_keep_prob)
    d_cell_bw1 = tf.contrib.rnn.DropoutWrapper(cell_bw1, input_keep_prob=input_keep_prob)
    
    (fw_h1, bw_h1), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw1, d_cell_bw1, Wx, sequence_length=x_len, dtype='float', scope='lstm1')
    h1 = tf.concat([fw_h1, bw_h1], 2)
    
with tf.variable_scope("lstm2"):
    cell_fw2 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
    cell_bw2 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

    d_cell_fw2 = tf.contrib.rnn.DropoutWrapper(cell_fw2, input_keep_prob=input_keep_prob)
    d_cell_bw2 = tf.contrib.rnn.DropoutWrapper(cell_bw2, input_keep_prob=input_keep_prob)
    
    (fw_h2, bw_h2), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw2, d_cell_bw2, h1, sequence_length=x_len, dtype='float', scope='lstm2')
    h = tf.concat([fw_h2, bw_h2], 2)
    
def attention(input_x, input_mask, W_att):
    h_masked = tf.boolean_mask(input_x, input_mask)
    h_tanh = tf.tanh(h_masked)
    u = tf.matmul(h_tanh, W_att)
    a = tf.nn.softmax(u)
    c = tf.reduce_sum(tf.multiply(h_tanh, a), 0)  
    return c

with tf.variable_scope("attention"):
    W_att = tf.Variable(tf.truncated_normal([2*hidden_size, 1], -0.1, 0.1), name="W_att")
    c = tf.expand_dims(attention(h[0], x_mask[0], W_att), 0)
    for i in range(1, batch_size):
        c = tf.concat([c, tf.expand_dims(attention(h[i], x_mask[i], W_att), 0)], 0)
        
with tf.variable_scope("softmax_layer"):
    W = tf.Variable(tf.truncated_normal([2*hidden_size, num_senses], -0.1, 0.1), name="W")
    b = tf.Variable(tf.zeros([num_senses]), name="b")
    drop_c = tf.nn.dropout(c, input_keep_prob)
    logits = tf.matmul(drop_c, W) + b
    predictions = tf.argmax(logits, 1)

loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
global_step = tf.Variable(0, trainable=False, name="global_step")

learning_rate = tf.train.exponential_decay(init_lr, global_step, decay_steps, decay_rate, staircase=True)

tv_all = tf.trainable_variables()
tv_regu =[]
for t in tv_all:
    if t.name.find('b:')==-1:
        tv_regu.append(t)
        
# l2 Loss
l2_loss = l2_lambda * tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv_regu ])

total_loss = loss + l2_loss

# Optimizer for loss
optimizer = tf.train.AdamOptimizer(learning_rate)

# Gradients and Variables for Loss
grads_vars = optimizer.compute_gradients(total_loss)

# Clipping of Gradients
clipped_grads = grads_vars
if(clipping == True):
    clipped_grads = [(tf.clip_by_norm(grad, clip_norm), var) for grad, var in clipped_grads]

# Training Optimizer for Total Loss
train_op = optimizer.apply_gradients(clipped_grads, global_step=global_step)

# Summaries
var_summaries = []
for v in tv_all:
    var_summary = tf.summary.histogram("{}/var".format(v.name), v)
    var_summaries.append(var_summary)

var_summaries_merged = tf.summary.merge(var_summaries)

loss_summary = tf.summary.scalar("loss", loss)
total_loss_summary = tf.summary.scalar("total_loss", total_loss)
summary = tf.summary.merge_all()

In [6]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"
print (device_lib.list_local_devices())
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
sess.run(tf.global_variables_initializer())                          # For initializing all the variables
saver = tf.train.Saver()                                             # For Saving the model
summary_writer = tf.summary.FileWriter(log_dir, sess.graph)          # For writing Summaries

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11746204887754786772
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 7958282240
locality {
  bus_id: 1
}
incarnation: 7822182935895831615
physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:06:00.0"
]


In [7]:
# Splitting
data_x = full_data[sense_word][0]
data_y = full_data[sense_word][2]
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, train_size=0.8, shuffle=True, stratify=data_y)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.9, shuffle=True, stratify=y_train)

/users/btech/aviraj/cs771/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [8]:
def data_prepare(x):
    num_examples = len(x)

    xx = np.zeros([num_examples, max_sent_size], dtype=int)
    xx_mask = np.zeros([num_examples, max_sent_size], dtype=bool)

    for j in range(num_examples):
        for i in range(max_sent_size):
            if(i>=len(x[j])):
                break
            w = x[j][i]
            xx[j][i] = word2id[w] if w in word2id else word2id['UNKNOWN_TOKEN']
            xx_mask[j][i] = True
            
    return xx, xx_mask

def eval_f1(yy, pred):
    num_batches = int(len(yy)/batch_size)
    f1 = f1_score(yy[:batch_size*num_batches], pred, average='macro')
    return f1*100

def model(xx, yy, mask, train_cond=True):
    num_batches = int(len(xx)/batch_size)
    losses = 0
    preds = []
    for j in range(num_batches): 
        
        s = j * batch_size
        e = (j+1) * batch_size
        
        feed_dict = {x:xx[s:e], y:yy[s:e], x_mask:mask[s:e], is_train:train_cond, input_keep_prob:keep_prob, word_emb_mat:word_embedding}
        
        
        if(train_cond==True):
            _, _loss, step, _summary = sess.run([train_op, total_loss, global_step, summary], feed_dict)
            summary_writer.add_summary(_summary, step)  
#             print("Steps:{}".format(step), ", Loss: {}".format(_loss))

        else:
            _loss, pred = sess.run([total_loss, predictions], feed_dict)
            preds.append(pred)
            
        losses +=_loss

    if(train_cond==False):
        y_pred = []
        for i in range(num_batches):
            for pred in preds[i]:
                y_pred.append(pred)
        return losses/num_batches, y_pred
    
    return losses/num_batches, step

In [9]:
x_id_train, mask_train = data_prepare(x_train)
x_id_val, mask_val = data_prepare(x_val)

In [10]:
num_epochs = 10

for i in range(num_epochs):
    
#     random = np.random.choice(num_train_data, size=(num_train_data), replace=False)
#     X_train = train_x[random]
#     Y_train = train_y[random]
#     X_mask_train = train_x_mask[random]
            
    losses, step = model(x_id_train, y_train, mask_train)
    print("Epoch:", i+1,"Step:", step, "loss:",losses)
    saver.save(sess, save_path=save_dir) 
    val_loss, val_pred = model(x_id_val, y_val, mask_val, train_cond=False)
    print("Validation F1 Score: ",  eval_f1(y_val, val_pred), "Loss: ", val_loss)

Epoch: 1 Step: 48 loss: 1.82507693271
Validation F1 Score:  44.9995104159 Loss:  1.42079758644
Epoch: 2 Step: 96 loss: 1.2814425615
Validation F1 Score:  49.9930381509 Loss:  1.20273165703
Epoch: 3 Step: 144 loss: 1.09923300644
Validation F1 Score:  49.4996186117 Loss:  1.09550886154
Epoch: 4 Step: 192 loss: 0.989965242644
Validation F1 Score:  48.617838302 Loss:  0.960019719601
Epoch: 5 Step: 240 loss: 0.909283029536
Validation F1 Score:  53.8547054277 Loss:  0.914924633503
Epoch: 6 Step: 288 loss: 0.845369558781
Validation F1 Score:  52.1149351076 Loss:  0.807123339176
Epoch: 7 Step: 336 loss: 0.744119742264
Validation F1 Score:  55.0871459695 Loss:  0.840722203255
Epoch: 8 Step: 384 loss: 0.729566873362
Validation F1 Score:  54.9949216714 Loss:  0.700761437416
Epoch: 9 Step: 432 loss: 0.691164808969
Validation F1 Score:  50.993618682 Loss:  0.701508486271
Epoch: 10 Step: 480 loss: 0.646685534467
Validation F1 Score:  46.7383909364 Loss:  0.722565531731


In [11]:
num_epochs = 20

for i in range(num_epochs):
    
#     random = np.random.choice(num_train_data, size=(num_train_data), replace=False)
#     X_train = train_x[random]
#     Y_train = train_y[random]
#     X_mask_train = train_x_mask[random]
            
    losses, step = model(x_id_train, y_train, mask_train)
    print("Epoch:", i+1,"Step:", step, "loss:",losses)
    saver.save(sess, save_path=save_dir) 
    val_loss, val_pred = model(x_id_val, y_val, mask_val, train_cond=False)
    print("Validation F1 Score: ",  eval_f1(y_val, val_pred), "Loss: ", val_loss)

Epoch: 1 Step: 528 loss: 0.599114766344
Validation F1 Score:  63.7427600613 Loss:  0.636759734154
Epoch: 2 Step: 576 loss: 0.59020898367
Validation F1 Score:  59.2336746228 Loss:  0.592612588406
Epoch: 3 Step: 624 loss: 0.57848552304
Validation F1 Score:  61.5212066833 Loss:  0.58812404871
Epoch: 4 Step: 672 loss: 0.546057891101
Validation F1 Score:  68.0498733684 Loss:  0.521329689026
Epoch: 5 Step: 720 loss: 0.524260375649
Validation F1 Score:  58.0252247017 Loss:  0.605982613564
Epoch: 6 Step: 768 loss: 0.524811490749
Validation F1 Score:  63.9523406059 Loss:  0.588197565079
Epoch: 7 Step: 816 loss: 0.503076875582
Validation F1 Score:  66.8098512132 Loss:  0.538520765305
Epoch: 8 Step: 864 loss: 0.481807698806
Validation F1 Score:  68.5078497109 Loss:  0.495990926027
Epoch: 9 Step: 912 loss: 0.472864597415
Validation F1 Score:  74.1994295465 Loss:  0.450396043062
Epoch: 10 Step: 960 loss: 0.458657657728
Validation F1 Score:  75.815425421 Loss:  0.48773342371
Epoch: 11 Step: 1008 los

In [12]:
num_epochs = 20

for i in range(num_epochs):
    
#     random = np.random.choice(num_train_data, size=(num_train_data), replace=False)
#     X_train = train_x[random]
#     Y_train = train_y[random]
#     X_mask_train = train_x_mask[random]
            
    losses, step = model(x_id_train, y_train, mask_train)
    print("Epoch:", i+1,"Step:", step, "loss:",losses)
    saver.save(sess, save_path=save_dir) 
    val_loss, val_pred = model(x_id_val, y_val, mask_val, train_cond=False)
    print("Validation F1 Score: ",  eval_f1(y_val, val_pred), "Loss: ", val_loss)

Epoch: 1 Step: 1488 loss: 0.38899099951
Validation F1 Score:  75.9709737465 Loss:  0.427084571123
Epoch: 2 Step: 1536 loss: 0.373093746292
Validation F1 Score:  79.4888681642 Loss:  0.395564982295
Epoch: 3 Step: 1584 loss: 0.380437946257
Validation F1 Score:  75.1474840949 Loss:  0.420517444611
Epoch: 4 Step: 1632 loss: 0.362508803296
Validation F1 Score:  74.975247719 Loss:  0.49052143693
Epoch: 5 Step: 1680 loss: 0.350373744965
Validation F1 Score:  75.48583139 Loss:  0.428182834387
Epoch: 6 Step: 1728 loss: 0.351464365919
Validation F1 Score:  77.5921855922 Loss:  0.41224014163
Epoch: 7 Step: 1776 loss: 0.373582904538
Validation F1 Score:  83.6125356125 Loss:  0.354309561849
Epoch: 8 Step: 1824 loss: 0.354874616799
Validation F1 Score:  80.7114378009 Loss:  0.436035090685
Epoch: 9 Step: 1872 loss: 0.357729084169
Validation F1 Score:  80.7178508514 Loss:  0.386241966486
Epoch: 10 Step: 1920 loss: 0.345169805301
Validation F1 Score:  77.6387101363 Loss:  0.397305491567
Epoch: 11 Step:

In [13]:
num_epochs = 20

for i in range(num_epochs):
    
#     random = np.random.choice(num_train_data, size=(num_train_data), replace=False)
#     X_train = train_x[random]
#     Y_train = train_y[random]
#     X_mask_train = train_x_mask[random]
            
    losses, step = model(x_id_train, y_train, mask_train)
    print("Epoch:", i+1,"Step:", step, "loss:",losses)
    saver.save(sess, save_path=save_dir) 
    val_loss, val_pred = model(x_id_val, y_val, mask_val, train_cond=False)
    print("Validation F1 Score: ",  eval_f1(y_val, val_pred), "Loss: ", val_loss)

Epoch: 1 Step: 2448 loss: 0.289533912515
Validation F1 Score:  82.4211611427 Loss:  0.394497048855
Epoch: 2 Step: 2496 loss: 0.295340486492
Validation F1 Score:  82.1832268261 Loss:  0.367476922274
Epoch: 3 Step: 2544 loss: 0.302833015099
Validation F1 Score:  84.2710301275 Loss:  0.399037802219
Epoch: 4 Step: 2592 loss: 0.29110698433
Validation F1 Score:  82.5519099203 Loss:  0.377544307709
Epoch: 5 Step: 2640 loss: 0.286863371109
Validation F1 Score:  85.1641089024 Loss:  0.327539241314
Epoch: 6 Step: 2688 loss: 0.293681959311
Validation F1 Score:  79.0461778351 Loss:  0.415607535839
Epoch: 7 Step: 2736 loss: 0.286511713328
Validation F1 Score:  83.3679225202 Loss:  0.472890973091
Epoch: 8 Step: 2784 loss: 0.288319438696
Validation F1 Score:  81.7070538486 Loss:  0.401738473773
Epoch: 9 Step: 2832 loss: 0.272951453303
Validation F1 Score:  85.6136656938 Loss:  0.323090258241
Epoch: 10 Step: 2880 loss: 0.268453218353
Validation F1 Score:  85.9271650249 Loss:  0.327444496751
Epoch: 11 